In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm

class SMT_test:
    chart = None
    def __init__(self, benchmarks_path='../../', chart_path='../charts', result_path='../results'):
        self.benchmarks_path = benchmarks_path
        self.chart_path = chart_path
        self.result_path = result_path

    def init_chart(self, folder_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_folder_path = f'{self.benchmarks_path}{folder_path}'
        self.get_info(full_folder_path)
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)
    
    def get_info(self, folder_path):
        benchmarks = os.listdir(folder_path)
        for file in benchmarks:
            path = os.path.join(folder_path, file)
            if os.path.isdir(path):
                self.get_info(path)
            else:
                try:
                    with open(path) as f:
                        vars = 0
                        assertions = 0
                        status = 'none'
                        line = f.readline()
                        while line:
                            if line[:3] == '(de':
                                vars += 1
                            elif line[:3] == '(as':
                                assertions += 1
                            elif line[11:17] == 'status':
                                status = line.split(' ')[2][:-2]
                            line = f.readline()
                        category = path.split('/')[-2]
                        path = path.replace(self.benchmarks_path, '')
                        self.chart.loc[len(self.chart.index)] = [path, category, vars, assertions, status]
                except:
                    print(path)

    def arrange(self, solver, log_path, read_path, save_path):
        full_execl_path = f'{self.chart_path}/{read_path}'
        self.chart = pd.read_excel(full_execl_path, header=[0], index_col = [0])
        full_res_path = f'{self.result_path}/{solver}/{log_path}'
        results = os.listdir(full_res_path)
        result_name = f'result_{solver}_{log_path}'
        time_name = f'time_{solver}_{log_path}'
        total = len(self.chart.index)
        count = 0
        for file in tqdm(results, desc=full_res_path):
            with open(os.path.join(full_res_path, file)) as f:
                try:
                    if solver == 'z3' or solver == 'z3pp_ls' or solver == 'z3pp':
                        while f.readline():
                            result = f.readline().rstrip()
                            # OMT-test only
                            if result == "sat" or result == "unknown":
                                f.readline()
                                f.readline()
                                f.readline()
                            elif result[:3] == "./s":
                                result = "unknown"
                            info = f.readline().rstrip()
                            if info == "unknown":
                                info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'cvc5':
                        while f.readline():
                            info = f.readline().rstrip()
                            result = info
                            if info == 'unknown':
                                info = f.readline().rstrip()
                                if len(info) < 10:
                                    if len(info) > 1:
                                        result = info
                                    info = f.readline().rstrip()
                            elif len(info) > 10:
                                result = "unknown"
                            else:
                                info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'cvc5_new':
                        while f.readline():
                            info = f.readline().rstrip()
                            result = info
                            if len(info) > 10:
                                result = "unknown"
                            info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'yices2':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == 'QF_':
                                info = result
                                result = 'timeout'
                            else:
                                info = f.readline()[:-1]
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'mathsat5':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == './s':
                                result = 'timeout'
                            if result[:3] == 'tcm':
                                result = 'timeout'
                                info = f.readline()
                                while info[:3] == 'tcm':
                                    info = f.readline()
                            if result[:3] == '(er':
                                result = 'not support'
                            info = f.readline()[:-1]
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'smtrat':
                        while f.readline():
                            result = f.readline().rstrip()
                            if result[:3] == './s':
                                result = 'timeout'
                            if result[:3] == 'QF_':
                                info = result
                                result = 'none'
                            else:
                                info = f.readline()[:-1]
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                    elif solver == 'optimathsat':
                        while f.readline():
                            info = f.readline().rstrip()
                            if len(info) > 10:
                                if info[:3] == 'OMT':
                                    result = "timeout"
                                else:
                                    result = "unknown"
                                    info = f.readline().rstrip()
                            else:
                                result = info
                                info = f.readline().rstrip()
                            name = info.split(' : ')[0]
                            timeout = info.split(' : ')[1][:-4]
                            self.chart.loc[self.chart['filename'] == name, [result_name, time_name]] = [result, float(timeout)]
                            count += 1
                except:
                    print(file)

        print(f"count/total: {count}/{total}")
        
        full_save_path = f'{self.chart_path}/{save_path}'
        self.chart.to_excel(full_save_path)

In [2]:
# str = SMT_test()
# str.init_chart('QF_S', 'base.xlsx', 'QF_S.xlsx')
# str.init_chart('QF_SLIA', 'base.xlsx', 'QF_SLIA.xlsx')
# str.arrange('cvc5', 'QF_S', 'QF_S.xlsx', 'result_QF_S.xlsx')
# str.arrange('cvc5', 'QF_SLIA', 'QF_SLIA.xlsx', 'result_QF_SLIA.xlsx')
# str.arrange('cvc5_new', 'QF_SLIA_unknown', 'QF_SLIA.xlsx', 'result_QF_SLIA_unknown.xlsx')
# str.arrange('cvc5_new', 'QF_S_unknown', 'QF_S.xlsx', 'result_QF_S_unknown.xlsx')

In [3]:
# omt = SMT_test()
# str.init_chart('OMT_LIA', 'base.xlsx', 'OMT_LIA.xlsx')
# omt.arrange('optimathsat', 'OMT_LIA', 'OMT_LIA.xlsx', 'result_optimathsat_LIA.xlsx')

In [4]:
omt = SMT_test()
omt.arrange('z3', 'OMT_LIA_1', 'OMT_LIA.xlsx', 'result_OMT_LIA.xlsx')

../results/z3/OMT_LIA_1:   0%|          | 0/166 [00:00<?, ?it/s]

count/total: 829/8034


In [5]:
omt = SMT_test()
omt.arrange('z3pp', 'OMT_LIA', 'result_OMT_LIA.xlsx', 'result_OMT_LIA.xlsx')

../results/z3pp/OMT_LIA:   0%|          | 0/166 [00:00<?, ?it/s]

count/total: 829/8034


In [6]:
omt = SMT_test()
omt.arrange('z3pp_ls', 'OMT_LIA_1', 'result_OMT_LIA.xlsx', 'result_OMT_LIA.xlsx')

../results/z3pp_ls/OMT_LIA_1:   0%|          | 0/166 [00:00<?, ?it/s]

count/total: 829/8034
